In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="18036c9769a64a9488a17419863beb3b" 
secret = "6dfa96c4f860468f8bf4a796e2b90df6"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [7]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
spotify_href = []
images = []
release_date = []

for i in range(0,10000,50):
    track_results = sp.search(q='year:2005', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
        spotify_href.append(t['external_urls']['spotify'])
        try:
            images.append(t['album']['images'][0]['url'])
        except:
            images.append('')
        release_date.append(t['album']['release_date'])
            
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

SSLError: HTTPSConnectionPool(host='api.spotify.com', port=443): Max retries exceeded with url: /v1/search?q=year%3A2005&limit=50&offset=6450&type=track (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')")))

In [4]:
track_results['tracks']['items'][1]['album']['images'][0]['url']

'https://i.scdn.co/image/09488edb691315df75090584328e095efd9b2576'

In [5]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 10000


In [6]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity,
                         'url':spotify_href,'images':images,'date':release_date})
print(df_tracks.shape)
df_tracks.head()

(10000, 7)


,artist_name,track_name,track_id,popularity,url,images,date
0,Panic! At The Disco,I Write Sins Not Tragedies,4bPQs0PHn4xbipzdPfn6du,76,https://open.spotify.com/track/4bPQs0PHn4xbipz...,https://i.scdn.co/image/76c86f33b0ffc852e4141e...,2005-09-27
1,Eminem,"Lose Yourself - From ""8 Mile"" Soundtrack",5Z01UMMf7V1o0MzF86s6WJ,80,https://open.spotify.com/track/5Z01UMMf7V1o0Mz...,https://i.scdn.co/image/77eb7c17cafe5503fc12e9...,2005-12-06
2,Fall Out Boy,"Sugar, We're Goin Down",2TfSHkHiFO4gRztVIkggkE,78,https://open.spotify.com/track/2TfSHkHiFO4gRzt...,https://i.scdn.co/image/63a37f32006eab70bd58c8...,2005-01-01
3,Kanye West,Gold Digger,1PS1QMdUqOal0ai3Gt7sDQ,78,https://open.spotify.com/track/1PS1QMdUqOal0ai...,https://i.scdn.co/image/92db1dda6ed503723c89c3...,2005-09-30
4,Plain White T's,Hey There Delilah,4RCWB3V8V0dignt99LZ8vH,79,https://open.spotify.com/track/4RCWB3V8V0dignt...,https://i.scdn.co/image/3a3e50577a99bfca93494b...,2005-01-01


In [7]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
artist_name    10000 non-null object
track_name     10000 non-null object
track_id       10000 non-null object
popularity     10000 non-null int64
url            10000 non-null object
images         10000 non-null object
date           10000 non-null object
dtypes: int64(1), object(6)
memory usage: 547.0+ KB


In [8]:
 # group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

376

In [9]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [10]:
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [11]:
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()

artist_name    0
track_name     0
track_id       0
popularity     0
url            0
images         0
date           0
dtype: int64

In [12]:
df_tracks.shape

(9599, 7)

In [13]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 31.137054060999986


## 트랙의 다른 유용한 데이터 찾기

In [14]:
# 트랙의 장르 값 찾아보기
feature_results

[{'danceability': 0.352,
  'energy': 0.319,
  'key': 0,
  'loudness': -17.635,
  'mode': 1,
  'speechiness': 0.0409,
  'acousticness': 0.882,
  'instrumentalness': 0.879,
  'liveness': 0.405,
  'valence': 0.436,
  'tempo': 176.439,
  'type': 'audio_features',
  'id': '4xbhOd2MLaqBylpk2Q4UF9',
  'uri': 'spotify:track:4xbhOd2MLaqBylpk2Q4UF9',
  'track_href': 'https://api.spotify.com/v1/tracks/4xbhOd2MLaqBylpk2Q4UF9',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4xbhOd2MLaqBylpk2Q4UF9',
  'duration_ms': 424200,
  'time_signature': 3},
 {'danceability': 0.853,
  'energy': 0.726,
  'key': 4,
  'loudness': -5.538,
  'mode': 0,
  'speechiness': 0.0564,
  'acousticness': 0.00431,
  'instrumentalness': 0.591,
  'liveness': 0.765,
  'valence': 0.662,
  'tempo': 120.506,
  'type': 'audio_features',
  'id': '57QCMbXNiEwurLPprfD3Gq',
  'uri': 'spotify:track:57QCMbXNiEwurLPprfD3Gq',
  'track_href': 'https://api.spotify.com/v1/tracks/57QCMbXNiEwurLPprfD3Gq',
  'analysis_url': 'https:/

In [5]:
# 새로운 노래
sp.new_releases()

{'albums': {'href': 'https://api.spotify.com/v1/browse/new-releases?offset=0&limit=20',
  'items': [{'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2auiVi8sUZo17dLy1HwrTU'},
      'href': 'https://api.spotify.com/v1/artists/2auiVi8sUZo17dLy1HwrTU',
      'id': '2auiVi8sUZo17dLy1HwrTU',
      'name': 'Solange',
      'type': 'artist',
      'uri': 'spotify:artist:2auiVi8sUZo17dLy1HwrTU'}],
    'available_markets': ['AD',
     'AE',
     'AR',
     'AT',
     'AU',
     'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DE',
     'DK',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FR',
     'GB',
     'GR',
     'GT',
     'HK',
     'HN',
     'HU',
     'ID',
     'IE',
     'IL',
     'IN',
     'IS',
     'IT',
     'JO',
     'JP',
     'KW',
     'LB',
     'LI',
     'LT',
     'LU',
     'LV',
     'MA',
    